In [1]:
# Function to one-hot encode the dataset
def encode_transactions(df):
    items = set()
    for transaction in df.values.flatten():
        if isinstance(transaction, str):
            items.add(transaction)
    
    # Convert items set to a list for columns
    items = list(items)
    
    # Create one-hot encoded dataframe
    one_hot = pd.DataFrame(columns=items, index=df.index).fillna(0)
    
    for idx, row in df.iterrows():
        for item in row:
            if pd.notna(item):
                one_hot.at[idx, item] = 1
    return one_hot
